In [44]:
import csv
from gmplot import gmplot
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [14]:
gmplot.GoogleMapPlotter.from_geocode("New York")

IndexError: list index out of range

In [18]:
def represents_int(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [38]:
# Goal:
# 262 EAST 55 STREET, BROOKLYN NY 11203 =>
# 262 EAST 55th STREET, BROOKLYN NY 11203

ENDINGS = {
    '0': 'th',
    '1': 'st',
    '2': 'nd',
    '3': 'rd',
    '4': 'th',
    '5': 'th',
    '6': 'th',
    '7': 'th',
    '6': 'th',
    '8': 'th',
    '9': 'th'
}
def convert_street(addr):
    # e.g. 
    (street, rest) = addr.split(',', 1)
    str_tokens = street.split(' ')
    for i in range(len(str_tokens)):
        if i == 0: continue
        t = str_tokens[i]
        if represents_int(t):
            str_tokens[i] += ENDINGS[t[-1]]
    new_street = ' '.join(str_tokens)
    return new_street + ',' + rest

In [39]:
convert_street('483 EAST 180 STREET, BRONX NY 10457')

'483 EAST 180th STREET, BRONX NY 10457'

In [50]:
gl = Nominatim(user_agent='hri_0', timeout=10)

In [52]:
count = 0
points = []
to_latlong = {}
with open('to_lat_long_1.csv', 'w') as out:
    writer = csv.DictWriter(out, fieldnames=[
        'id','geo_addr','count','lat','long'
    ])
    with open('lead_paint_processed.csv', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            count += 1
            if count < 500:
                continue
            
            addr = convert_street(row['address'])
            try:
                loc = gl.geocode(addr)
            except GeocoderTimedOut as e:
                print('Geo Time Out: ' + e)
                continue
            if not loc:
                print('Geo error: ' + row['building_id'] + ' | ' + addr)
                continue
            
            latlong = (loc.latitude, loc.longitude)
            writer.writerow({'id': row['building_id'],
                             'geo_addr': addr,
                             'count': row['num_violations'],
                             'lat': latlong[0],
                             'long': latlong[1]})
            
            for i in range(int(row['num_violations'])):
                points.append(latlong)

            if count % 100 == 0:
                print(str(count) + ' processed.')

500 processed.
Geo error: 334598 | 44 MAC DONOUGH STREET, BROOKLYN NY 11216
Geo error: 932167 | 2006 CROSS BRONX SERVICE ROAD SOUTH, BRONX NY 10472
Geo error: 812538 | 260 PARKHILL AVENUE, STATEN ISLAND NY 10304
Geo error: 116845 | 1805 DR M L KING JR BOULEVARD, BRONX NY 10453
Geo error: 111430 | 1302 REV JAMES POLITE AVENUE, BRONX NY 10459
Geo error: 812535 | 160 PARKHILL AVENUE, STATEN ISLAND NY 10304
600 processed.
Geo error: 3376 | 1845 ADAM C POWELL BOULEVARD, MANHATTAN NY 10026
Geo error: 335113 | 314 MAC DOUGAL STREET, BROOKLYN NY 11233
Geo error: 209757 | 1819 BEVERLY ROAD, BROOKLYN NY 11226
700 processed.
Geo error: 116950 | 2352 DR M L KING JR BOULEVARD, BRONX NY 10468
Geo error: 3634 | 2530 ADAM C POWELL BOULEVARD, MANHATTAN NY 10039
800 processed.
Geo error: 67818 | 664 EAST 213rd STREET, BRONX NY 10467
Geo error: 116925 | 2265 DR M L KING JR BOULEVARD, BRONX NY 10468
900 processed.
1000 processed.
Geo error: 116881 | 1970 DR M L KING JR BOULEVARD, BRONX NY 10453
Geo error:

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests